In [0]:
from keras import backend as K
from keras.layers import Convolution2D, ZeroPadding2D, Activation, Input, concatenate, Dropout
from keras import models
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.core import Lambda, Flatten, Dense
model = models.Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
#model.add(Convolution2D(1000, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [0]:
!unzip v4_SplitData.zip

Archive:  v4_SplitData.zip
replace v4_SplitData/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: [y]
error:  invalid response [[y]]
replace v4_SplitData/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: v4_SplitData/.DS_Store  
replace __MACOSX/v4_SplitData/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/v4_SplitData/._.DS_Store  
replace v4_SplitData/test/ramtej cl4/IMG-20190702-WA1682.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
from keras.models import model_from_json
model.load_weights('vgg_face_weights.h5')

W0708 09:20:01.541143 140114672977792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [0]:
from keras.models import Model
vgg_face_descriptor = Model(inputs=model.layers[0].input
, outputs=model.layers[-2].output)

In [0]:
model.layers[-4]

In [0]:
import numpy as np
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import preprocess_input

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

In [0]:
def findCosineDistance(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))
def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [0]:
epsilon = 0.40 #cosine similarity
#epsilon = 120 #euclidean distance
 
def verifyFace(img1, img2):
  img1_representation = vgg_face_descriptor.predict(preprocess_image(img1))[0,:]
  img2_representation = vgg_face_descriptor.predict(preprocess_image(img2))[0,:]
  
  cosine_similarity = findCosineDistance(img1_representation, img2_representation)
  euclidean_distance = findEuclideanDistance(img1_representation, img2_representation)
 
  if(cosine_similarity < epsilon):
    print("verified... they are same person")
  else:
    print("unverified! they are not same person!")

In [0]:
img1 = 'jb-20-390x285.jpg'
img2 = 'jb3.jpeg'

In [0]:
#img1_representation = vgg_face_descriptor.predict(preprocess_image('17ANGELINA-JOLIE1-articleLarge.jpg'))[0,:]
#img2_representation = vgg_face_descriptor.predict(preprocess_image('Angelina-Jolie-Nato-t.jpg'))[0,:]

In [0]:
verifyFace(img1, img2)

verified... they are same person


In [0]:
from keras.models import model_from_json
model.load_weights('vgg_face_weights.h5')

In [0]:
from keras.models import Model
vgg_face_descriptor = Model(inputs=model.layers[0].input
, outputs=model.layers[-2].output)

In [0]:
model.layers[-4]

**For vgg face fine_tuning**



In [0]:
height = 224
width = 224

In [0]:
vgg_face = vgg_face_descriptor

In [0]:
# Freeze the layers except the last 8 layers
for layer in vgg_face.layers[:-16]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_face.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f938ee31860> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f938ee317b8> False
<keras.layers.convolutional.Conv2D object at 0x7f938ee317f0> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f938ee31b00> False
<keras.layers.convolutional.Conv2D object at 0x7f938ee31f60> False
<keras.layers.pooling.MaxPooling2D object at 0x7f938edc1828> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f938ee31940> False
<keras.layers.convolutional.Conv2D object at 0x7f938edc1eb8> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f938ee1da90> False
<keras.layers.convolutional.Conv2D object at 0x7f938edf0400> False
<keras.layers.pooling.MaxPooling2D object at 0x7f938ed83cc0> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f938edf02e8> False
<keras.layers.convolutional.Conv2D object at 0x7f938ed830f0> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f938ed83b70> False
<keras.layers.conv

In [0]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_face)
 
# Add new layers
#model.add(layers.Dense(1024, activation='relu'))
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(55, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_5 (Model)              (None, 2622)              145002878 
_________________________________________________________________
dense_5 (Dense)              (None, 55)                144265    
Total params: 145,147,143
Trainable params: 139,871,687
Non-trainable params: 5,275,456
_________________________________________________________________


In [0]:
from keras.preprocessing.image import ImageDataGenerator
train_dir = 'v4_SplitData/train'
validation_dir = 'v4_SplitData/test'
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 100
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(height, width),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(height, width),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 884 images belonging to 55 classes.
Found 247 images belonging to 55 classes.


In [0]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=70,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
 
# Save the model
model.save('small_last4_vgg_face5.h5')

Epoch 1/70
9/8 [==============================] - 32s 4s/step - loss: 4.0117 - acc: 0.0330 - val_loss: 3.9810 - val_acc: 0.0769
Epoch 2/70
9/8 [==============================] - 26s 3s/step - loss: 4.2435 - acc: 0.0464 - val_loss: 3.4418 - val_acc: 0.1093
Epoch 3/70
9/8 [==============================] - 27s 3s/step - loss: 3.6568 - acc: 0.0971 - val_loss: 3.3323 - val_acc: 0.1215
Epoch 4/70
9/8 [==============================] - 27s 3s/step - loss: 3.3954 - acc: 0.1322 - val_loss: 2.8186 - val_acc: 0.1700
Epoch 5/70
9/8 [==============================] - 27s 3s/step - loss: 3.0115 - acc: 0.1754 - val_loss: 3.0277 - val_acc: 0.1700
Epoch 6/70
9/8 [==============================] - 27s 3s/step - loss: 2.7615 - acc: 0.2298 - val_loss: 1.6416 - val_acc: 0.5223
Epoch 7/70
9/8 [==============================] - 27s 3s/step - loss: 2.3404 - acc: 0.3457 - val_loss: 1.3517 - val_acc: 0.5992
Epoch 8/70
9/8 [==============================] - 27s 3s/step - loss: 1.8029 - acc: 0.4671 - val_loss: 4